In [1]:
from dotenv import load_dotenv
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np 
from Preprocessing.preprocessing_pipeline_initial import preprocessing_pipeline
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

load_dotenv()  # lädt .env automatisch
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")



In [2]:
# Standardmäßig werden alle Zeilen angezeigt, egal, wie viele es sind
pd.set_option("display.max_columns", None)

In [ ]:
import concurrent.futures

def call_llm(text):
    # Diese Funktion nutzt die API von Google um sich gegen das LLM zu schalten. Prompt etc. ist unten zu sehen

    llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001", #
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''Du bist ein Experte für Automobilklassifikation. Deine Aufgabe ist es, Automodelle basierend auf ihrem Namen einem von fünf vordefinierten Fahrzeugsegmenten zuzuordnen. 
                    Bitte gib nur den Namen des passenden Segments zurück.
                    Die Segmente sind:
                    1. Kleinwagen – kleine Stadtautos, z.B. VW Up!, Renault Clio, Fiat Panda
                    2. Mittelklasse – normale Alltagsautos, z.B. VW Golf, Audi A4, BMW 3er
                    3. Geländewagen – große Fahrzeuge mit viel Platz, z.B. BMW X3, VW Tiguan, Ford Kuga, VW Multivan
                    4. Sportwagen – sportliche Fahrzeuge mit viel PS, z.B. Porsche 911, Audi R8, BMW M4
                    5. Luxusklasse – hochwertige Fahrzeuge mit Premiumausstattung, z.B. BMW 7er, Mercedes S-Klasse, Tesla Model S
                    Beispiele:

                    Input: smart forTwo
                    Output: Kleinwagen
                    Input: Volkswagen Golf
                    Output: Mittelklasse
                    Input: BMW X5
                    Output: Geländewagen
                    Input: Porsche 911
                    Output: Sportwagen
                    Input: Bentley Mulsanne
                    Output: Luxusklasse
                Gib mir als Output nur das Fahrzeugsegment aus.                          
                ''',
            ),
            ("human", "{input}"),
        ]
    )

    chain = prompt | llm
    res = chain.invoke(
        {
            "input": text
        }
    )

    return res.content



def use_llm_on_model_parallel(model_list, max_workers=5):
    result = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_model = {executor.submit(call_llm, model): model for model in model_list}
        
        for future in concurrent.futures.as_completed(future_to_model):
            model = future_to_model[future]
            try:
                result[model] = future.result()
            except Exception as e:
                result[model] = f"Error: {e}"

    return result
'''
model_list = df['model'].unique().tolist()
result = use_llm_on_model_parallel(model_list)
df["segment"] = df["model"].map(result).fillna("unknown")
'''

'\ndef use_llm_on_model(model_list):\n    # Diese Funktion ruft die call_llm Funktion auf, um der Spalte model das Fahrzeugsegment zuzuordnen\n\n    result = {}\n    counter = 0\n    limit_per_minute = 15 # brauchen diesen Timer um nicht zu viele Anfragen zu stellen (free sind 15 Anfragen/Min)\n\n    for idx, model in enumerate(model_list, start=1):\n        result[model] = call_llm(model)\n        counter += 1\n\n        # Warten, wenn das Limit erreicht ist\n        if counter % limit_per_minute == 0:\n            print(f"{counter} Anfragen gestellt – warte 60 Sekunden, um Rate Limit einzuhalten...")\n            time.sleep(60)\n\n    print("Fertig!")\n    return result\n\ndef use_llm_on_model(model_list):\n    # Diese Funktion ruft die call_llm Funktion auf, um der Spalte model das Fahrzeugsegment zuzuordnen\n\n    result = {}\n\n    for model in (model_list):\n        result[model] = call_llm(model)\n\n    return result\n'

In [ ]:
df = preprocessing_pipeline()

In [ ]:
import concurrent.futures
import functools # Wird benötigt, um Argumente an die Funktion im Thread zu binden

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage


# Deine 8 finalen Segmente
segments = [
    "Kleinwagen",
    "Kompaktklasse",
    "Mittelklasse",
    "Obere Mittelklasse",
    "Oberklasse/Luxusklasse",
    "SUV",
    "Van",
    "Sportwagen"
]
VALID_SEGMENTS_UPDATED = segments # Für die Validierung

# Neuer System-Prompt (gekürzt zur Übersicht, nimm den vollständigen von oben)
system_prompt = f'''Du bist ein Experte für Automobilklassifikation in Deutschland. Deine Aufgabe ist es, Automodelle (inkl. Marke) einem der folgenden {len(segments)} vordefinierten Fahrzeugsegmenten zuzuordnen.
Bitte gib nur den Namen des passenden Segments zurück. Stelle sicher, dass deine Antwort EXAKT einem der unten genannten Segmente entspricht.

Die Segmente und Beispiele sind:

1.  Kleinwagen: Kleine Stadtflitzer und Superminis.
     z.B.: VW Polo, Opel Corsa, Ford Fiesta, Renault Clio, Peugeot 208, Toyota Yaris, Skoda Fabia, Fiat 500, Mini Cooper, Hyundai i20, Seat Ibiza

2.  Kompaktklasse: Die "Golf-Klasse", untere Mittelklasse.
     z.B.: VW Golf, Audi A3, BMW 1er, Mercedes A-Klasse, Opel Astra, Ford Focus, Seat Leon, Skoda Octavia, Hyundai i30, Kia Ceed

3.  Mittelklasse: Standard-Limousinen und Kombis für Familie/Beruf.
     z.B.: VW Passat, Audi A4, BMW 3er, Mercedes C-Klasse, Skoda Superb, Ford Mondeo, Opel Insignia, Volvo S60/V60, Tesla Model 3

4.  Obere Mittelklasse: Größere Geschäfts- und Reisefahrzeuge.
     z.B.: Audi A6, BMW 5er, Mercedes E-Klasse, Volvo S90/V90, Jaguar XF, Lexus ES

5.  Oberklasse/Luxusklasse: Repräsentative Luxusfahrzeuge.
     z.B.: Audi A8, BMW 7er, Mercedes S-Klasse, Porsche Panamera, Lexus LS, Bentley Continental GT, Tesla Model S

6.  SUV: Sport Utility Vehicles, verschiedene Größen, oft mit erhöhter Sitzposition.
    z.B.: VW Tiguan, VW Touareg, Audi Q3/Q5/Q7, BMW X1/X3/X5, Mercedes GLA/GLC/GLE, Ford Kuga, Skoda Kodiaq/Karoq, Porsche Macan/Cayenne, Volvo XC60, Dacia Duster

7.  Van: Familienvans, Kleinbusse mit viel Platz.
     z.B.: VW Touran, VW Sharan, VW Multivan/Caravelle, Mercedes V-Klasse/Vito Tourer, Ford Galaxy/S-Max, Opel Zafira, Seat Alhambra, Renault Espace

8.  Sportwagen: Leistungsstarke Coupés, Roadster oder Sportlimousinen.
     z.B.: Porsche 911, Porsche 718 Cayman/Boxster, Audi R8, Audi TT, BMW M2/M3/M4, Mercedes-AMG GT/SL, Ford Mustang, Jaguar F-Type, Nissan GT-R

Gib mir als Output NUR das EINE passende Fahrzeugsegment aus der Liste {segments} zurück. KEINEN zusätzlichen Text.
'''

def setup_llm_chain_updated():
    """Initialisiert das LLM und die Prompt Chain mit den Segmenten."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-001", # anderes Modell noch testen
        temperature=0, # Kreativität
        max_tokens=50, #länge der Antwort (kurz da nur Segment erwartet)
        timeout=None,
        max_retries=2
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"), # {input} sollte "Marke Modell" sein
        ]
    )
    chain = prompt | llm
    return chain

# --- 2. Die Funktion, die EINEN EINZELNEN LLM-Aufruf macht (angepasst) ---
# Diese Funktion wird von den parallelen Threads aufgerufen.
# Sie bekommt jetzt die vorkonfigurierte Chain übergeben.

def call_llm_single_validated(model_input_string, chain_instance):
    """
    Ruft die LLM-Chain für einen einzelnen Modell-String auf und validiert das Ergebnis.

    Args:
        model_input_string: Der Input für das LLM (Format "Marke Modell").
        chain_instance: Die vorkonfigurierte LangChain-Chain.

    Returns:
        Das validierte Segment (String) oder eine Fehlermeldung (String).
    """
    try:
        # Rufe die Chain für den einzelnen Input auf
        response = chain_instance.invoke({"input": model_input_string})

        if isinstance(response, AIMessage):
            segment = response.content.strip()
            # Validierung gegen die erlaubten Segmente
            if segment in VALID_SEGMENTS_UPDATED:
                return segment
            else:
                # LLM hat ein ungültiges Segment zurückgegeben
                print(f"Warning: Invalid segment '{segment}' received for '{model_input_string}'")
                return f"Error: Invalid Segment - '{segment}'" # Oder 'Sonstige'/None
        else:
            # Unerwarteter Rückgabetyp
            print(f"Warning: Unexpected output type {type(response)} for '{model_input_string}'")
            return f"Error: Unexpected Output Type"

    except Exception as e:
        # Fehler während des LLM-Aufrufs selbst
        print(f"Error calling LLM for '{model_input_string}': {e}")
        return f"Error: API Call Failed - {e}"

# --- 3. Deine parallele Ausführungsfunktion (leicht angepasst) ---
# Sie muss die vorkonfigurierte Chain erhalten und an call_llm_single_validated übergeben.

def use_llm_on_model_parallel_updated(model_strings_list, chain_to_use, max_workers=5):
    """
    Nutzt ThreadPoolExecutor, um call_llm_single_validated parallel für eine Liste
    von Modell-Strings aufzurufen.

    Args:
        model_strings_list: Liste von Strings (Format "Marke Modell").
        chain_to_use: Die vorkonfigurierte LangChain-Chain, die verwendet werden soll.
        max_workers: Maximale Anzahl paralleler Threads.

    Returns:
        Ein Dictionary, das Input-Strings auf Ergebnisse abbildet.
    """
    results = {}

    # functools.partial wird verwendet, um die 'chain_to_use' fest an
    # call_llm_single_validated zu binden. Der Executor übergibt dann nur noch
    # das jeweilige 'model_string' aus der Liste.
    func_to_call = functools.partial(call_llm_single_validated, chain_instance=chain_to_use)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Sende Jobs an den Executor
        # Map Input-String zum Future-Objekt
        future_to_model = {executor.submit(func_to_call, model_str): model_str for model_str in model_strings_list}

        # Sammle Ergebnisse, sobald sie fertig sind
        for future in concurrent.futures.as_completed(future_to_model):
            model_str = future_to_model[future]
            try:
                # Hole das Ergebnis vom Future (dies kann auch eine Exception auslösen,
                # falls call_llm_single_validated eine Exception wirft, was wir aber
                # durch try/except innerhalb der Funktion meist vermeiden)
                result_segment = future.result()
                results[model_str] = result_segment
            except Exception as exc:
                # Falls doch eine unerwartete Exception aus future.result() kommt
                print(f'Model {model_str} generated an exception: {exc}')
                results[model_str] = f"Error: Exception - {exc}"

    return results

# --- 4. Beispiel-Anwendung ---

# WICHTIG: Stelle sicher, dass deine Input-Liste Strings im Format "Marke Modell" enthält!
# Beispiel:
# model_list = ["Volkswagen Golf", "Porsche 911", "Mercedes-Benz G 63 AMG", "Fiat 500", "BMW X5", "Ford Mustang"]
# Ersetze dies durch deine tatsächliche Liste von Modell-Strings
unique_model_brand_strings = list(df[['brand', 'model']].apply(lambda x: f"{x['brand']} {x['model']}", axis=1).unique())

# a) Erstelle die LLM-Chain EINMALIG
print("Setting up LLM chain...")
my_llm_chain = setup_llm_chain_updated()
print("LLM chain ready.")

# b) Rufe die parallele Funktion auf und übergib die Chain
print(f"Starting parallel classification for {len(unique_model_brand_strings)} unique models...")
classification_results = use_llm_on_model_parallel_updated(
    model_strings_list=unique_model_brand_strings,
    chain_to_use=my_llm_chain,
    max_workers=10 # Passe die Worker-Anzahl nach Bedarf an
)
print("Parallel classification finished.")

# c) Ergebnisse anzeigen (Auszug)
print("\nSample Results:")
print(dict(list(classification_results.items())[:10]))

# d) Ergebnisse zurück in den DataFrame mappen (Beispiel)
print("\nMapping results back to DataFrame...")
df['segment'] = df.apply(lambda x: classification_results.get(f"{x['brand']} {x['model']}", "Error: Mapping Failed"), axis=1)
print(df[['brand', 'model', 'segment']].head())
print(f"\nValue counts for segments:\n{df['segment'].value_counts()}")

Setting up LLM chain...
LLM chain ready.
Starting parallel classification for 343 unique models...
Parallel classification finished.

Sample Results:
{'audi Audi TT': 'Sportwagen', 'audi Audi A8': 'Oberklasse/Luxusklasse', 'audi Audi S6': 'Obere Mittelklasse', 'audi Audi A4': 'Mittelklasse', 'audi Audi A3': 'Kompaktklasse', 'audi Audi A6': 'Obere Mittelklasse', 'alfa-romeo Alfa Romeo Giulietta': 'Kompaktklasse', 'alfa-romeo Alfa Romeo Stelvio': 'SUV', 'audi Audi S3': 'Kompaktklasse', 'audi Audi A6 allroad': 'Obere Mittelklasse'}


In [ ]:
df.to_csv('df_mit_segmenten.csv', index= False)